In [1]:
from PyPDF2 import PdfMerger, PageRange
import os
from natsort import os_sorted
import pandas as pd
import numpy as np

def flatten(t):
    return [item for sublist in t for item in sublist]

def Display(in_list):
    for i in in_list:
        print(i)
    return None

directory_root = 'C:\\Users\\kitsu\\Documents\\0TLCSD\\eMATHinstruction\\'

def Solutions(in_truth=False):
    if bool(in_truth) == False:
        return 'Problems\\'
    else:
        return 'Solutions\\'

curriculum = 'N-Gen_Math'
courses = ['7', '8', 'Algebra_I']

units0 = ['1', '2', '3', '4', '5', '6', '7', '7cc', '8', '8cc', '9', '10']
units1 = ['1', '2', '3', '4', '5', '6', '6cc', '7', '8', '9', '10', '11', '12cc']
units2 = [str(i) for i in range(1,12)]
units = [units0, units1, units2]

directories = []
for truth in [False, True]:
    for course in courses:
        directories.append(directory_root + Solutions(truth) + curriculum+'_'+course+'\\')

files = []
for directory in directories:
    files.append(os_sorted(list(os.walk(directory))[0][2]))
files[3] = files[3][1:]
# print(files[3])
    
def Check_Files(in_files):
    check0 = []
    check1 = []
    check2 = []
    check3 = []
    check4 = []
    check5 = []
    check  = [check0, check1, check2, check3, check4, check5]
    capture = []
    for truth in [False, True]:
        for course in courses:
            if course == 'Algebra_I':
                alt_course = 'Algebra-I'
            else:
                alt_course = course
            #
            t = int(truth)
            c = int(courses.index(course))
            i = len(courses)*t + c
            #
            i_UNITS = [x.replace('.Unit-', '.').split('.')[1] for x in files[i]]
            i_units = pd.Series(i_UNITS).unique().tolist()
            i_u_counts = pd.Series(i_UNITS).value_counts(sort=False).tolist()
            i_u_indices = [sum(i_u_counts[:i]) for i in range(1+len(i_u_counts))]
            capture.append(i_u_indices)
            #
            i_check = check[i]
            i_files = files[i]
            # Folder only contains the correct course
            i_check.append(i_files == [x for x in i_files if 'Math-' + alt_course in x])
            # Folder contains the correct units
            i_check.append(i_units == units[c])
            # Folder only contains the correct answer key
            if truth == False:
                i_check.append(i_files == [x for x in i_files if 'Answer-Key' not in x])
            else:
                i_check.append(i_files == [x for x in i_files if 'Answer-Key' in x])
            # For each unit, our lessons are 1, 2, ..., end
            for j in range(len(i_u_counts)):
                j_l = [x.replace('Lesson-', '.').split('.')[3] for x in i_files[i_u_indices[j]:i_u_indices[j+1]]]
                i_check.append(j_l == [str(i) for i in range(1,i_u_counts[j]+1)])
            bad = [x for x in check if False in x]
    if len(bad) == 0:
        print('Looks good!')
    else:
        print('Trouble at', check.index(bad[0]), ':', bad[0].index(False), "O'Clock!")
    return [capture[0], capture[1], capture[2], check]

J = Check_Files(files)
# Display(J)





FILES = []
for truth in [False, True]:
    for c in range(3):
        t = bool(truth)
        i = t*len(courses)+c
        for j in range(len(files[i])):
            FILES.append(directories[i]+files[i][j])
# Display(FILES)
# print('\n'*2)

JJ = J[0].copy()
jj = JJ[-1]
for j in J[1][1:]:
    JJ.append(jj+j)
jj=JJ[-1]
for j in J[2][1:]:
    JJ.append(jj+j)
h = len(JJ)
jj=JJ[-1]
for i in range(len(JJ)-1):
    JJ.append(jj+JJ[i+1])
# print(JJ)

IND = [j for j in range(len(FILES)-1) if FILES[j].split('.')[0][-1] != FILES[j+1].split('.')[0][-1]]
# print(IND)

# for i in range(len(JJ)-1):
#     print(i, JJ[i], JJ[i+1], FILES[JJ[i]:JJ[i+1]][0], FILES[JJ[i]:JJ[i+1]][-1], '\n', sep='\n')





IND0 = [x+1 for x in IND]
IND0.append(0)
IND0 = sorted(IND0)
IND0.append(JJ[-1])
# print(IND0)
# print('\n'*2)

# for i in range(len(IND0)-1):
#     print(i, IND0[i], IND0[i+1], FILES[IND0[i]:IND0[i+1]][0], FILES[IND0[i]:IND0[i+1]][-1], '\n', sep='\n')
    # Display(FILES[IND0[i]:IND0[i+1]])
    # print('\n')









merger = PdfMerger()

for i in range(len(JJ)-1):
    #
    if i < h:
        A = ''
    else:
        A='_ANSWERS'
    #
    X = '\\'.join(FILES[JJ[i]].split('\\')[:-1]) + '\\'
    # XX = '\\'.join(FILES[JJ[i]].split('\\')[-2:-1]) + '\\'
    Y = FILES[JJ[i]].split('Unit-')[1].split('.')[0]
    # print(X, XX, Y)
    #
    for f in FILES[JJ[i]:JJ[i+1]]:
        # print(f)
        merger.append(f)
    merger.write(X + '10_' + 'UNIT_' + Y + A + '.pdf')
    merger = PdfMerger()
    #
    for f in FILES[JJ[i]:JJ[i+1]]:
        merger.append(f, pages=PageRange(':2'))
    merger.write(X + '11_' + 'UNIT_' + Y + '_MATERIAL' + A + '.pdf')
    merger = PdfMerger()
    #
    for f in FILES[JJ[i]:JJ[i+1]]:
        merger.append(f, pages=PageRange('2:'))
    merger.write(X + '12_' + 'UNIT_' + Y + '_HW' + A + '.pdf')
    merger = PdfMerger()


merger = PdfMerger()

for i in range(len(IND0)-1):
    #
    if i < 3:
        A = ''
    else:
        A = '_ANSWERS'
    #
    X = '\\'.join(FILES[IND0[i]].split('\\')[:-1]) + '\\'
    # XX = '\\'.join(FILES[IND0[i]].split('\\')[-1]) + '\\'
    Y = courses[int(i%3)].upper()
    #
    for f in FILES[IND0[i]:IND0[i+1]]:
        merger.append(f)
    merger.write(X + '00_' + 'COURSE_MATH_' + Y + A + '.pdf')
    merger = PdfMerger()
    #
    for f in FILES[IND0[i]:IND0[i+1]]:
        merger.append(f, pages=PageRange(':2'))
    merger.write(X + '01_' + 'COURSE_MATH_' + Y + '_MATERIAL' + A + '.pdf')
    merger = PdfMerger()
    #
    for f in FILES[IND0[i]:IND0[i+1]]:
        merger.append(f, pages=PageRange('2:'))
    merger.write(X + '02_' + 'COURSE_MATH_' + Y + '_HW' + A + '.pdf')
    merger = PdfMerger()

Looks good!
0
0
8
C:\Users\kitsu\Documents\0TLCSD\eMATHinstruction\Problems\N-Gen_Math_7\N-Gen-Math-7.Unit-1.Lesson-1.Multiplication-1.pdf
C:\Users\kitsu\Documents\0TLCSD\eMATHinstruction\Problems\N-Gen_Math_7\N-Gen-Math-7.Unit-1.Lesson-8.Working-with-Rates-Using-the-Calculator-1.pdf


1
8
20
C:\Users\kitsu\Documents\0TLCSD\eMATHinstruction\Problems\N-Gen_Math_7\N-Gen-Math-7.Unit-2.Lesson-1.The-Rational-Numbers-1.pdf
C:\Users\kitsu\Documents\0TLCSD\eMATHinstruction\Problems\N-Gen_Math_7\N-Gen-Math-7.Unit-2.Lesson-12.Working-with-Signed-Numbers-on-the-Calculator-1.pdf


2
20
30
C:\Users\kitsu\Documents\0TLCSD\eMATHinstruction\Problems\N-Gen_Math_7\N-Gen-Math-7.Unit-3.Lesson-1.Ratios-1.pdf
C:\Users\kitsu\Documents\0TLCSD\eMATHinstruction\Problems\N-Gen_Math_7\N-Gen-Math-7.Unit-3.Lesson-10.More-Work-with-Proportional-Relationships-1.pdf


3
30
40
C:\Users\kitsu\Documents\0TLCSD\eMATHinstruction\Problems\N-Gen_Math_7\N-Gen-Math-7.Unit-4.Lesson-1.Finding-Fractions-of-Quantities-1.pdf
C:\Use